In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
def compare_all_metrics_bootstrap(returns_A, returns_B, strategy_A_name="Strategy A", strategy_B_name="Strategy B", n_bootstrap=1000):
    """
    Compara un conjunto de métricas clave entre dos estrategias usando block bootstrap
    para determinar la significancia estadística de las diferencias.
    """
    
    # --- Funciones para calcular cada métrica a partir de una serie de retornos ---
    def get_total_return(r):
        return (1 + r).prod() - 1
    
    def get_cagr(r):
        n_years = len(r) / 365.25
        if n_years == 0: return 0
        return (1 + get_total_return(r))**(1/n_years) - 1

    def get_annualized_vol(r):
        return r.std() * np.sqrt(365)

    def get_max_drawdown(r):
        portfolio_values = (1 + r).cumprod()
        rolling_max = portfolio_values.cummax()
        drawdown = portfolio_values / rolling_max - 1
        return drawdown.min()

    def get_sharpe(r):
        if get_annualized_vol(r) == 0: return 0
        # Usamos el retorno promedio diario * 365 para el retorno anualizado
        return (r.mean() * 365) / get_annualized_vol(r)

    def get_sortino_ratio(r):
        downside_returns = r[r < 0]
        downside_std = downside_returns.std() * np.sqrt(365)
        if downside_std == 0: return 0
        return (r.mean() * 365) / downside_std

    def get_calmar_ratio(r):
        mdd = get_max_drawdown(r)
        if mdd == 0: return 0
        return get_cagr(r) / abs(mdd)

    # --- Inicialización ---
    metrics_to_test = {
        "Total Return": get_total_return,
        "CAGR": get_cagr,
        "Annualized Vol": get_annualized_vol,
        "Max Drawdown": get_max_drawdown,
        "Sharpe Ratio": get_sharpe,
        "Sortino Ratio": get_sortino_ratio,
        "Calmar Ratio": get_calmar_ratio
    }
    
    
    bootstrap_diffs = {key: [] for key in metrics_to_test}
    
    # --- Loop de Bootstrap ---
    block_size = int(np.floor(len(returns_A)**(1/4)))
    
    for _ in range(n_bootstrap):
        # Crear muestras bootstrap de los retornos
        sample_A, sample_B = [], []
        num_blocks = int(np.ceil(len(returns_A) / block_size))
        for _ in range(num_blocks):
            start = np.random.randint(0, len(returns_A) - block_size + 1)
            sample_A.extend(returns_A.iloc[start : start + block_size])
            sample_B.extend(returns_B.iloc[start : start + block_size])
        
        sample_A = pd.Series(sample_A[:len(returns_A)])
        sample_B = pd.Series(sample_B[:len(returns_B)])

        # Calcular la diferencia para cada métrica y guardarla
        for name, func in metrics_to_test.items():
            metric_A = func(sample_A)
            metric_B = func(sample_B)
            bootstrap_diffs[name].append(metric_A - metric_B)

    # --- Reportar Resultados ---
    print(f"--- Comparación de Métricas: {strategy_A_name} vs. {strategy_B_name} ---\n")
    alpha = 0.05
    results_list = []
    
    for name, diffs in bootstrap_diffs.items():
        diffs = np.array(diffs)
        observed_diff = metrics_to_test[name](returns_A) - metrics_to_test[name](returns_B)
        
        # P-valor de una cola (H1: Métrica A > Métrica B)
        # Para Max Drawdown y Volatilidad, un valor MENOR es mejor, por lo que la lógica se invierte.
        if name in ["Max Drawdown", "Annualized Vol"]:
            p_value = (diffs >= 0).mean() # H1: Métrica A < Métrica B
            conclusion_text = f"La diferencia es estadísticamente significativa y a favor de {strategy_A_name} (menor es mejor).\n"
            is_significant = p_value < alpha and observed_diff < 0
        else:
            p_value = (diffs <= 0).mean() # H1: Métrica A > Métrica B
            conclusion_text = f"La diferencia es estadísticamente significativa y a favor de {strategy_A_name}.\n"
            is_significant = p_value < alpha and observed_diff > 0

        ci_lower = np.percentile(diffs, 2.5)
        ci_upper = np.percentile(diffs, 97.5)
        
        print(f"Métrica: {name}")
        print(f"  - Diferencia Observada: {observed_diff:.4f}")
        print(f"  - P-valor (Bootstrap): {p_value:.4f}")
        print(f"  - Intervalo de Confianza 95%: [{ci_lower:.4f}, {ci_upper:.4f}]")
        
        if is_significant:
            print(f"  - Conclusión: {conclusion_text}")
            final_conclusion = conclusion_text.strip()
        else:
            print(f"  - Conclusión: No hay evidencia estadística suficiente para afirmar una diferencia significativa.\n")
            final_conclusion = "No hay evidencia estadística suficiente para afirmar una diferencia significativa."

        results_list.append({
            "Metric": name,
            "Observed Difference": observed_diff,
            "P-value (Bootstrap)": p_value,
            "95% CI Lower": ci_lower,
            "95% CI Upper": ci_upper,
            "Is Significant": is_significant,
            "Conclusion": final_conclusion
        })
    return results_list

In [3]:
# --- Ejecutar las pruebas completas ---
# Cargar datos
portfolios_df = pd.read_csv('reports/strategies_portfolio.csv', parse_dates=['date'])
returns_df = portfolios_df.set_index('date').pct_change().dropna()

# Comparar Walk-Forward vs. Buy & Hold
results_bh = compare_all_metrics_bootstrap(
    returns_df['walk_forward_portfolio'], 
    returns_df['buy_and_hold_portfolio'],
    "Walk-Forward",
    "Buy & Hold"
)
df_bh = pd.DataFrame(results_bh)
df_bh.insert(0, 'Comparison', 'Walk-Forward vs. Buy & Hold')

print("\n" + "="*80 + "\n")

# Comparar Walk-Forward vs. DCA
results_dca = compare_all_metrics_bootstrap(
    returns_df['walk_forward_portfolio'], 
    returns_df['dca_portfolio'],
    "Walk-Forward",
    "DCA"
)
df_dca = pd.DataFrame(results_dca)
df_dca.insert(0, 'Comparison', 'Walk-Forward vs. DCA')

# Combinar y exportar a CSV
final_df = pd.concat([df_bh, df_dca], ignore_index=True)
output_path = 'reports/results/bootstrap_comparison_results.csv'
final_df.to_csv(output_path, index=False)

print(f"\nResultados de la comparación exportados a: {output_path}")

--- Comparación de Métricas: Walk-Forward vs. Buy & Hold ---

Métrica: Total Return
  - Diferencia Observada: 0.0292
  - P-valor (Bootstrap): 0.5180
  - Intervalo de Confianza 95%: [-1.1899, 0.8374]
  - Conclusión: No hay evidencia estadística suficiente para afirmar una diferencia significativa.

Métrica: CAGR
  - Diferencia Observada: 0.0414
  - P-valor (Bootstrap): 0.5180
  - Intervalo de Confianza 95%: [-1.8100, 1.2361]
  - Conclusión: No hay evidencia estadística suficiente para afirmar una diferencia significativa.

Métrica: Annualized Vol
  - Diferencia Observada: -0.1333
  - P-valor (Bootstrap): 0.0000
  - Intervalo de Confianza 95%: [-0.1949, -0.0804]
  - Conclusión: La diferencia es estadísticamente significativa y a favor de Walk-Forward (menor es mejor).

Métrica: Max Drawdown
  - Diferencia Observada: 0.0377
  - P-valor (Bootstrap): 0.8780
  - Intervalo de Confianza 95%: [-0.0458, 0.2311]
  - Conclusión: No hay evidencia estadística suficiente para afirmar una diferencia s